In [20]:
import os
import pandas as pd

IMAGE_DIR = r"C:\Users\maila\Desktop\Defect_Detection\Normalised_Image_256"
OLD_CSV = r"C:\Users\maila\Desktop\Defect_Detection\train.csv"
NEW_CSV = "train_clean.csv"

# Load old CSV (labels source)
df_old = pd.read_csv(OLD_CSV, dtype={"ID": str})

# Get available image IDs from folder
image_ids = {
    os.path.splitext(f)[0]
    for f in os.listdir(IMAGE_DIR)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
}

# Keep only rows whose images exist
df_clean = df_old[df_old["ID"].isin(image_ids)].reset_index(drop=True)

df_clean.to_csv(NEW_CSV, index=False)

print("Original CSV size:", len(df_old))
print("Clean CSV size   :", len(df_clean))
print("Removed rows     :", len(df_old) - len(df_clean))
print("Saved as:", NEW_CSV)


Original CSV size: 5701
Clean CSV size   : 5701
Removed rows     : 0
Saved as: train_clean.csv


In [21]:
df = pd.read_csv("train_clean.csv", dtype={"ID": str})

df_ok = df[df["label"] == 1].sample(1000, random_state=42)
df_not_ok = df[df["label"] == 0].sample(1000, random_state=42)

df_balanced = (
    pd.concat([df_ok, df_not_ok])
    .sample(frac=1, random_state=42)
    .reset_index(drop=True)
)

df_balanced.to_csv("train_1000_clean.csv", index=False)

print("Balanced CSV saved: train_2000_clean.csv")


Balanced CSV saved: train_2000_clean.csv


In [22]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor()   # converts PIL image → Tensor [0,1]
])


In [23]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
import torch

class DefectDataset(Dataset):
    def __init__(self, csv_path, image_dir, transform=None):
        self.df = pd.read_csv(csv_path, dtype={"ID": str})
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx]["ID"]
        label = int(self.df.iloc[idx]["label"])

        for ext in (".jpg", ".jpeg", ".png"):
            img_path = os.path.join(self.image_dir, img_id + ext)
            if os.path.exists(img_path):
                break
        else:
            raise FileNotFoundError(f"Image not found for ID: {img_id}")

        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)


In [24]:
dataset = DefectDataset(
    csv_path="train_1000_clean.csv",
    image_dir=r"C:\Users\maila\Desktop\Defect_Detection\Normalised_Image_256",
    transform=transform
)


In [25]:
from torch.utils.data import DataLoader, random_split
BALANCED_CSV = "train_1000_clean.csv"

IMAGE_DIR = r"C:\Users\maila\Desktop\Defect_Detection\Normalised_Image_256"

dataset = DefectDataset(BALANCED_CSV, IMAGE_DIR, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

In [26]:
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)


In [27]:
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 20

for epoch in range(EPOCHS):
    model.train()
    train_correct = 0
    train_total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        images = images.to(device)
        labels = labels.to(device).unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = (torch.sigmoid(outputs) > 0.5).int()
        train_correct += (preds == labels.int()).sum().item()
        train_total += labels.size(0)

    train_acc = train_correct / train_total

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device).unsqueeze(1)

            outputs = model(images)
            preds = (torch.sigmoid(outputs) > 0.5).int()
            val_correct += (preds == labels.int()).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total

    print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")


Epoch 1/20: 100%|██████████| 50/50 [00:26<00:00,  1.88it/s]


Epoch 1: Train Acc=0.5850, Val Acc=0.6425


Epoch 2/20: 100%|██████████| 50/50 [00:15<00:00,  3.13it/s]


Epoch 2: Train Acc=0.6400, Val Acc=0.6625


Epoch 3/20: 100%|██████████| 50/50 [00:16<00:00,  3.04it/s]


...

Epoch 18: Train Acc=1.0000, Val Acc=0.9775


Epoch 19/20: 100%|██████████| 50/50 [00:16<00:00,  2.96it/s]


Epoch 19: Train Acc=1.0000, Val Acc=0.9725


Epoch 20/20: 100%|██████████| 50/50 [00:16<00:00,  2.95it/s]


Epoch 20: Train Acc=0.9981, Val Acc=0.9700


In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import torch

model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).int().squeeze(1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

# Convert to numpy arrays
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# ================= METRICS =================
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

cm = confusion_matrix(all_labels, all_preds)

print("\n================ MODEL EVALUATION ================")
print(f"Accuracy  : {accuracy:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall    : {recall:.4f}")
print(f"F1 Score  : {f1:.4f}")

print("\nConfusion Matrix:")
print(cm)
print("=================================================")



================ MODEL EVALUATION ================
Accuracy  : 0.9700
Precision : 0.9662
Recall    : 0.9756
F1 Score  : 0.9709

Confusion Matrix:
[[188   7]
 [  5 200]]
